In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import sys
from pathlib import Path

# Zorg dat build_training_set geïmporteerd is
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

# Define feature columns and target
FEATURES = ['weekday_cos', 'weekday_sin', 'hour_cos', 'hour_sin', 'yearday_cos', 'yearday_sin']
TRAIN_FEATURES = FEATURES
target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing XGBoost Model - RMSE per forecast day")
print("=" * 60)

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime').set_index('target_datetime')

        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df.index <= run_date_utc]
        test_data = df[df.index > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=TRAIN_FEATURES + [target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Train model
        X_train = train_data[TRAIN_FEATURES]
        y_train = train_data[target]
        model = xgb.XGBRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Zorg dat alle features aanwezig zijn in test_data
        for col in TRAIN_FEATURES:
            if col not in test_data.columns:
                test_data[col] = np.nan

        X_test = test_data[TRAIN_FEATURES]
        y_test = test_data[target]

        # Sla de eerste 24 uur over
        X_test = X_test.iloc[24:]
        y_test = y_test.iloc[24:]

        # Alleen voorspellen als er nog testdata over is
        if len(X_test) > 0:
            y_pred = model.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            print("Niet genoeg testdata na lag van 24 uur.")
            rmse = np.nan

        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(X_test),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(X_test)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - XGBoost Model")
    print("=" * 80)
    print(f"Successful runs: {rmse_df['rmse'].notna().sum()}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.4f}")
    print(f"Stddev RMSE: {rmse_df['rmse'].std():.4f}")

else:
    print("❌ No runs completed successfully")

2025-05-30 15:46:19,422 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-15 00:00:00+00:00 for lagging support
2025-05-30 15:46:19,422 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-30 15:46:19,422 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00 (extended to 2025-03-15 00:00:00+00:00 for lagging)
2025-05-30 15:46:19,431 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, normalized to 2025-03-15 00:00:00+00:00 for DB lookup, target range: 2025-03-15 00:00:00+00:00 → 2025-03-22 00:00:00+00:00
2025-05-30 15:46:19,431 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-05-30 15:46:19,431 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-05-30 15:46:19,440 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'di

🔍 Testing XGBoost Model - RMSE per forecast day


2025-05-30 15:46:19,599 - build_training_set - INFO - ✅ Predictions loaded: 169 rows with 17 columns
2025-05-30 15:46:19,599 - build_training_set - INFO - 🔧 Missing columns in predictions: ['Price', 'Load', 'Flow_NO', 'Flow_GB']
2025-05-30 15:46:19,599 - build_training_set - INFO - 📊 Applying 168-hour lag for missing columns (excluding target variables)...
2025-05-30 15:46:19,607 - build_training_set - INFO -    🎯 Column 'Price' is target variable - filled with NaN (not lagged)
2025-05-30 15:46:19,608 - build_training_set - INFO -    🕐 Lagging column 'Load' by 168 hours
2025-05-30 15:46:19,698 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:46:19,698 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:46:19,780 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:46:19,780 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:46:19,849 - build_training_s

Day 1: ✅ 144 test rows, Run: 03-15


2025-05-30 15:46:20,436 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:46:20,507 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:46:20,507 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:46:20,584 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:46:20,592 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:46:20,592 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:46:20,598 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:46:20,636 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:46:20,636 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:46:20,645 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-

...

Day 28: ✅ 144 test rows, Run: 04-11


2025-05-30 15:46:38,273 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:46:38,279 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:46:38,421 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:46:38,421 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:46:38,540 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:46:38,546 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:46:38,546 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:46:38,554 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:46:38,622 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:46:38,622 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:46:38,630 - buil

Day 29: ✅ 144 test rows, Run: 04-12


2025-05-30 15:46:39,028 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:46:39,171 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:46:39,171 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:46:39,305 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:46:39,314 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:46:39,314 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:46:39,330 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:46:39,396 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:46:39,396 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:46:39,405 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-

Day 30: ✅ 144 test rows, Run: 04-13

📊 OVERALL RMSE - XGBoost Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                144  0.05
         2 2025-03-16                144  0.05
         3 2025-03-17                144  0.03
         4 2025-03-18                144  0.04
         5 2025-03-19                144  0.04
         6 2025-03-20                144  0.04
         7 2025-03-21                144  0.04
         8 2025-03-22                144  0.06
         9 2025-03-23                144  0.05
        10 2025-03-24                144  0.04
        11 2025-03-25                144  0.05
        12 2025-03-26                144  0.05
        13 2025-03-27                144  0.06
        14 2025-03-28                144  0.05
        15 2025-03-29                144  0.06
        16 2025-03-30                144  0.05
        17 2025-03-31                144  0.06
        18 2025-04-01                144  0.04
        19 2025-

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import sys
from pathlib import Path

# Zorg dat build_training_set geïmporteerd is
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

# Define feature columns and target
FEATURES = ['Load', 'shortwave_radiation', 'temperature_2m', 
            'direct_normal_irradiance', 'diffuse_radiation', 
            'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 
            'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 
            'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 
            'hour_sin', 'weekday_cos']
TRAIN_FEATURES = FEATURES
target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing XGBoost Model - RMSE per forecast day")
print("=" * 60)

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime').set_index('target_datetime')

        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df.index <= run_date_utc]
        test_data = df[df.index > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=TRAIN_FEATURES + [target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Train model
        X_train = train_data[TRAIN_FEATURES]
        y_train = train_data[target]
        model = xgb.XGBRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Zorg dat alle features aanwezig zijn in test_data
        for col in TRAIN_FEATURES:
            if col not in test_data.columns:
                test_data[col] = np.nan

        X_test = test_data[TRAIN_FEATURES]
        y_test = test_data[target]

        # Sla de eerste 24 uur over
        X_test = X_test.iloc[24:]
        y_test = y_test.iloc[24:]

        # Alleen voorspellen als er nog testdata over is
        if len(X_test) > 0:
            y_pred = model.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            print("Niet genoeg testdata na lag van 24 uur.")
            rmse = np.nan

        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(X_test),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(X_test)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - XGBoost Model")
    print("=" * 80)
    print(f"Successful runs: {rmse_df['rmse'].notna().sum()}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.4f}")
    print(f"Stddev RMSE: {rmse_df['rmse'].std():.4f}")

else:
    print("❌ No runs completed successfully")

2025-05-30 15:48:27,620 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-15 00:00:00+00:00 for lagging support
2025-05-30 15:48:27,629 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-30 15:48:27,631 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00 (extended to 2025-03-15 00:00:00+00:00 for lagging)
2025-05-30 15:48:27,633 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, normalized to 2025-03-15 00:00:00+00:00 for DB lookup, target range: 2025-03-15 00:00:00+00:00 → 2025-03-22 00:00:00+00:00
2025-05-30 15:48:27,634 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-05-30 15:48:27,645 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-05-30 15:48:27,648 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'di

🔍 Testing XGBoost Model - RMSE per forecast day


2025-05-30 15:48:27,812 - build_training_set - INFO - 🔧 Missing columns in predictions: ['Price', 'Load', 'Flow_NO', 'Flow_GB']
2025-05-30 15:48:27,812 - build_training_set - INFO - 📊 Applying 168-hour lag for missing columns (excluding target variables)...
2025-05-30 15:48:27,820 - build_training_set - INFO -    🎯 Column 'Price' is target variable - filled with NaN (not lagged)
2025-05-30 15:48:27,820 - build_training_set - INFO -    🕐 Lagging column 'Load' by 168 hours
2025-05-30 15:48:27,913 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:48:27,913 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:48:27,997 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:48:28,005 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:48:28,113 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:48:28,121 - build_training_set - I

Day 1: ✅ 144 test rows, Run: 03-15


2025-05-30 15:48:29,003 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:48:29,003 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:48:29,100 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:48:29,100 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:48:29,297 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:48:29,299 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:48:29,301 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:48:29,306 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:48:29,354 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:48:29,354 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:48:29,354 - buil

...

Day 28: ✅ 144 test rows, Run: 04-11


2025-05-30 15:48:52,550 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:48:52,550 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:48:52,682 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:48:52,682 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:48:52,808 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:48:52,817 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:48:52,817 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:48:52,826 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:48:52,897 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:48:52,897 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:48:52,905 - buil

Day 29: ✅ 144 test rows, Run: 04-12


2025-05-30 15:48:53,453 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:48:53,575 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:48:53,575 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:48:53,710 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:48:53,718 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:48:53,718 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:48:53,726 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:48:53,784 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:48:53,784 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:48:53,784 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-

Day 30: ✅ 144 test rows, Run: 04-13

📊 OVERALL RMSE - XGBoost Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                144  0.04
         2 2025-03-16                144  0.04
         3 2025-03-17                144  0.04
         4 2025-03-18                144  0.04
         5 2025-03-19                144  0.04
         6 2025-03-20                144  0.04
         7 2025-03-21                144  0.04
         8 2025-03-22                144  0.03
         9 2025-03-23                144  0.03
        10 2025-03-24                144  0.04
        11 2025-03-25                144  0.04
        12 2025-03-26                144  0.04
        13 2025-03-27                144  0.05
        14 2025-03-28                144  0.05
        15 2025-03-29                144  0.05
        16 2025-03-30                144  0.04
        17 2025-03-31                144  0.04
        18 2025-04-01                144  0.04
        19 2025-